<h1><center>Laboratorio N° 03: Pivoting y Grouping Sets

<h2>Parte 01. Escribir consultas que utilizan PIVOT.

1.1. Abrir la base de datos TSQL.

In [2]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00

1.2. Crear un vista de categorías de productos, sus cantidaddes por año.

In [5]:
CREATE VIEW Sales.CategoryQtyYear
AS
SELECT c.categoryname AS Category,
       SUM(od.qty) AS TotalQty, -- Suma de cantidades
       YEAR(o.orderdate) AS Orderyear
FROM Production.Categories AS c
INNER JOIN Production.Products AS p ON c.categoryid = p.categoryid
INNER JOIN Sales.OrderDetails AS od ON p.productid = od.productid
INNER JOIN Sales.Orders AS o ON od.orderid = o.orderid
GROUP BY c.categoryname, YEAR(o.orderdate); -- Agrupar por categoría y año
GO

: Msg 2714, Level 16, State 3, Procedure CategoryQtyYear, Line 1
Ya hay un objeto con el nombre 'CategoryQtyYear' en la base de datos.

Total execution time: 00:00:00.004

1.3. Probar la vista creada.

In [6]:
SELECT Category, TotalQty, Orderyear
FROM Sales.CategoryQtyYear;
GO

(24 rows affected)

Total execution time: 00:00:00.044

Category,TotalQty,Orderyear
Meat/Poultry,950,2006
Seafood,2716,2008
Dairy Products,4374,2007
Grains/Cereals,1377,2008
Confections,4137,2007
Produce,858,2008
Beverages,1842,2006
Condiments,1441,2008
Dairy Products,2689,2008
Seafood,3679,2007


1.4. Utilizar PIVOT entre categorías y años de ordenes.

In [7]:
SELECT Category, [2006], [2007], [2008]
FROM (SELECT Category, TotalQty, Orderyear FROM Sales.CategoryQtyYear) AS D
PIVOT(SUM(TotalQty) FOR Orderyear IN ([2006], [2007], [2008])) AS pvt
ORDER BY Category;
GO

(8 rows affected)

Total execution time: 00:00:00.017

Category,2006,2007,2008
Beverages,1842,3996,3694
Condiments,962,2895,1441
Confections,1357,4137,2412
Dairy Products,2086,4374,2689
Grains/Cereals,549,2636,1377
Meat/Poultry,950,2189,1060
Produce,549,1583,858
Seafood,1286,3679,2716


1.5. Para utilizar UNPIVOT, crear una tabla con el resultado de la consulta con PIVOT anterior.

In [9]:
CREATE TABLE [Sales].[PivotedCategorySales](
    [Category] [nvarchar](15) NOT NULL,
    [2006] [int] NULL,
    [2007] [int] NULL,
    [2008] [int] NULL
);
GO

INSERT INTO Sales.PivotedCategorySales (Category, [2006], [2007], [2008])
SELECT Category, [2006], [2007], [2008]
FROM (SELECT Category, TotalQty, Orderyear FROM Sales.CategoryQtyYear) AS D
PIVOT(SUM(TotalQty) FOR Orderyear IN ([2006], [2007], [2008])) AS p;
GO

: Msg 2714, Level 16, State 6, Line 1
Ya hay un objeto con el nombre 'PivotedCategorySales' en la base de datos.

(8 rows affected)

Total execution time: 00:00:00.015

1.6. Probrar la tabla generada

In [10]:
SELECT Category, [2006], [2007], [2008]
FROM Sales.PivotedCategorySales;
GO

(8 rows affected)

Total execution time: 00:00:00.004

Category,2006,2007,2008
Beverages,1842,3996,3694
Condiments,962,2895,1441
Confections,1357,4137,2412
Dairy Products,2086,4374,2689
Grains/Cereals,549,2636,1377
Meat/Poultry,950,2189,1060
Produce,549,1583,858
Seafood,1286,3679,2716


1.7. Utilizar UNPIVOT.

In [11]:
SELECT Category, [Value] AS Qty, Orderyear
FROM Sales.PivotedCategorySales
UNPIVOT([Value] FOR Orderyear IN ([2006], [2007], [2008])) AS unpvt;
GO

(24 rows affected)

Total execution time: 00:00:00.010

Category,Qty,Orderyear
Beverages,1842,2006
Beverages,3996,2007
Beverages,3694,2008
Condiments,962,2006
Condiments,2895,2007
Condiments,1441,2008
Confections,1357,2006
Confections,4137,2007
Confections,2412,2008
Dairy Products,2086,2006


1.8. Limpiar los cambios realizados.


In [12]:
IF OBJECT_ID('Sales.CategoryQtyYear', 'V') IS NOT NULL DROP VIEW Sales.CategoryQtyYear;
IF OBJECT_ID('Sales.PivotedCategorySales') IS NOT NULL DROP TABLE Sales.PivotedCategorySales;
GO

Commands completed successfully.

Total execution time: 00:00:00.003

<h2>Parte 02. Escribir consultas que utilizan UNPIVOT.

2.1. Ejecutar la siguiente vistas

2.2. Elaborar una consulta sin utilizar GROUPING SETS.

2.3. Consultar con GROUPING SETS.

2.4. Consultar con CUBE.

2.5. Consultar con ROLLUP

2.6. Utilizar Grouping_ID.

2.7. Finalmente limpiar los cambios

<h2>Ejercicio 03: Escribir consultas que usan las clausulas GROUPING SETS, CUBE y ROLLUP

3.1. Escribir una sentencia SELECT contra la tabla Sales.Customers y traer las columnas country, city
y una columna calculada noofcustomers como una cantidad de clientes. Traer multiples grouping
sets basados en las columnas country y city, la columna country, la columna city, y una columna con un grouping set vacio.

3.2. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Día de la columna orderdate como orderday.
<li>Valor de Total de Ventas utilizando la columna val como salesvalue.
<li>Devolver todas los posibles  grouping sets basados en las columnas orderyear, ordermonth, y orderday.

3.3. Copiar la consulta previa y modificarla para utilizar la clausula ROLLUP en vez de CUBE.

3.4. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Columna Calculada con el alias groupid (usar la función GROUPING_ID con los campos order year y order month como parametros).
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Valor Total de Ventas utilizando la columna val como salesvalue.
<li>Desde Año y mes month forman una jerarquia, retornar todos los grouping sets interesantes basados en las columnas orderyear y ordermonth y ordenar el resultado por groupid, orderyear, and ordermonth